In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import geopandas as gpd                                                                                                 #实现空间匹配
import matplotlib.pyplot as plt
import fiona as fna
from geopy.distance import geodesic as gd                                                                               #计算地理距离
from segregation.singlegroup import *
from segregation.multigroup import  *
from segregation import inference
import shapely.wkt

In [2]:
# Transform the original data into geodf
df = pd.read_csv('beijing_tianjin_seg_v2.csv', sep='\s+', encoding='utf-8') 
df.columns = ['city', 'county', 'border','highschool_and_below','college','undergraduate_and_above','2499andbelow','2500~3999','4000~7999','8000~19999','20000andabove']  
df_border = df['border'].str.split(',', expand=True)

border_combine = "POLYGON((" + df_border[0]+" "+df_border[1]+ "," +df_border[2]+" "+df_border[3]+ "," +df_border[4]+" "+df_border[5]+ "," +df_border[6]+" "+df_border[7]+","+df_border[8]+" "+df_border[9]+ "))"
border_combine=border_combine.tolist()

P=[]
for i in range(len(border_combine)):
    P.insert(i-1,shapely.wkt.loads(border_combine[i]))
p=gpd.GeoSeries(P)
df=df.drop(['border'], axis=1)
geo_df = gpd.GeoDataFrame(data=df,geometry=p)
geo_df.crs="epsg:3857"  #geo_df


In [3]:
geo_df.head

<bound method NDFrame.head of    city county  highschool_and_below  college  undergraduate_and_above  \
0   孝感市    云梦县                     9        1                        2   
1   安阳市    林州市                     6        0                        0   
2   定西市    通渭县                     1        0                        0   
3   宜宾市    长宁县                    65       10                        1   
4   百色市    田东县                   227       81                       13   
..  ...    ...                   ...      ...                      ...   
94  内江市    威远县                    88       11                        7   
95  定西市    临洮县                    47       15                        1   
96  常德市    桃源县                   114       27                        2   
97  衡阳市    衡阳县                     0        2                        0   
98  贺州市    八步区                   192       65                       12   

    2499andbelow  2500~3999  4000~7999  8000~19999  20000andabove  \
0           

The data is a sample for the number of population in each education group in granular units. We want to calculate segregation index in county level, and most of the counties has at least 1000 units.

In [4]:
"""
    Cauculate segregation index in county level
"""

df_code = pd.read_csv('county_list_merge.csv', sep='\s+', encoding='utf-8')     #read county list                                  
df_code.columns =['city', 'county', 'county_id']   
new_geodf = pd.merge(geo_df, df_code, how='inner', on=['city', 'county'])      # merge geodataframe with county list                                       
new_geodf['id'] = new_geodf.groupby(u'county_id', as_index=False).ngroup() 
new_geodf['edu_low']= new_geodf['highschool_and_below']    # change three groups into two groups, edu_low refers to the low education level group
new_geodf['income_low']=new_geodf['2499andbelow'] + new_geodf['2500~3999'] #calculate the number of population in low income group
new_geodf['poptotal']= new_geodf['highschool_and_below']+new_geodf['college']+new_geodf['undergraduate_and_above']  #calculate the number of population in each unit


In [5]:
new_geodf.head

<bound method NDFrame.head of    city county  highschool_and_below  college  undergraduate_and_above  \
0   孝感市    云梦县                     9        1                        2   
1   安阳市    林州市                     6        0                        0   
2   定西市    通渭县                     1        0                        0   
3   宜宾市    长宁县                    65       10                        1   
4   百色市    田东县                   227       81                       13   
..  ...    ...                   ...      ...                      ...   
94  内江市    威远县                    88       11                        7   
95  定西市    临洮县                    47       15                        1   
96  常德市    桃源县                   114       27                        2   
97  衡阳市    衡阳县                     0        2                        0   
98  贺州市    八步区                   192       65                       12   

    2499andbelow  2500~3999  4000~7999  8000~19999  20000andabove  \
0           

In [6]:
singleindexs = [Dissim,Gini,Entropy,Isolation,Atkinson,ModifiedDissim,
                ModifiedGini,BiasCorrectedDissim,SpatialDissim,BoundarySpatialDissim] #identify all indexs to be calculated
columnsname = ['Dissim','Gini','Entropy','Isolation','Atkinson','ModifiedDissim',
                'ModifiedGini','BiasCorrectedDissim','SpatialDissim','BoundarySpatialDissim']
testpar = ['evenness','systematic','evenness','evenness','evenness',
           'systematic','systematic','evenness','evenness','evenness'] #specify the inference parameter
indexnames = ['edu','income'] 

single_columns=[] #store the name of each index
for i in range(len(indexnames)):
    for j in range(len(columnsname)):
        single_columns.append(indexnames[i]+"_"+columnsname[j])
        proper_n = len(new_geodf['county_id'].unique())     
print(single_columns)

proper_list = new_geodf[["id","county_id"]].drop_duplicates().values

num_indexs=len(single_columns) 
cols=np.ones((proper_n,num_indexs*2))
output_np = np.c_[proper_list, cols]     # to record final output

['edu_Dissim', 'edu_Gini', 'edu_Entropy', 'edu_Isolation', 'edu_Atkinson', 'edu_ModifiedDissim', 'edu_ModifiedGini', 'edu_BiasCorrectedDissim', 'edu_SpatialDissim', 'edu_BoundarySpatialDissim', 'income_Dissim', 'income_Gini', 'income_Entropy', 'income_Isolation', 'income_Atkinson', 'income_ModifiedDissim', 'income_ModifiedGini', 'income_BiasCorrectedDissim', 'income_SpatialDissim', 'income_BoundarySpatialDissim']


### Simple Loop

This code sometimes come up with error “unsupported operand types(s) for +: ‘float’ and ‘’NoneType”. 
For 'sometimes' I mean this code doesn't always come up with error.

In [9]:
proper_df = pd.DataFrame()

for idnum in range(proper_n):  # avoid 0/0

    proper_df = new_geodf[new_geodf['id'] == idnum] #calculate for a specific county, while in this sample, we only have 1 unit in each county, so we treat all 99 records as a county, and use new_geodf for calculation below
    for j in range(len(singleindexs)):  
        edu_seg = singleindexs[j](new_geodf, group_pop_var='edu_low',total_pop_var='poptotal')
        output_np[idnum, 2 + j]= edu_seg.statistic
        income_seg = singleindexs[j](new_geodf, group_pop_var='income_low',total_pop_var='poptotal')
        output_np[idnum, 22 + j]= income_seg.statistic

AssertionError: Gaps in blk ref_locs

### Loop with ‘try’ module to avoid invalid interation

To solve the problem above, I added a 'try' module to update iteration until it is valid.

In [10]:
proper_df = pd.DataFrame()

for idnum in range(proper_n):  # avoid 0/0

    proper_df = new_geodf[new_geodf['id'] == idnum] #calculate for a specific county, while in this sample, we only have 1 unit in each county, so we treat all 99 records as a county, and use new_geodf for calculation below
    for j in range(len(singleindexs)):
        flag = 1
        while flag:
            try:
                edu_seg = singleindexs[j](new_geodf, group_pop_var='edu_low',total_pop_var='poptotal')
                output_np[idnum, 2 + j]= edu_seg.statistic
                income_seg = singleindexs[j](new_geodf, group_pop_var='income_low',total_pop_var='poptotal')
                output_np[idnum, 22 + j]= income_seg.statistic
                flag = 0
            except:
                flag = 1
        # edu 2~11,12~21,income1 22~31,32~41; income2 42~52,52~61

In [11]:
print(output_np)

[[4.60000000e+01 4.20923000e+05 1.53275349e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [3.80000000e+01 4.10581000e+05 1.53275349e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [8.90000000e+01 6.21121000e+05 1.53275349e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 ...
 [5.30000000e+01 4.30725000e+05 1.53275349e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [4.90000000e+01 4.30421000e+05 1.53275349e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [6.40000000e+01 4.51100000e+05 1.53275349e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]]


### Adding inference part to the loop with 'try'

When adding inference, it is time consuming. I didn't get a result for it yet.

In [ ]:
proper_df = pd.DataFrame()

for idnum in range(1):  # just consider 1 conuty

    proper_df = new_geodf[new_geodf['id'] == idnum] #calculate for a specific county, while in this sample, we only have 1 unit in each county, so we treat all 99 records as a county, and use new_geodf for calculation below
    for j in range(len(singleindexs)):
        flag = 1
        while flag:
            try:
                edu_seg = singleindexs[j](new_geodf, group_pop_var='edu_low',total_pop_var='poptotal')
                output_np[idnum, 2 + j]= edu_seg.statistic
                output_np[idnum, 12 + j]=inference.SingleValueTest(edu_seg, null_approach=testpar[j]).p_value           
                flag = 0
            except:
                flag = 1
        # edu 2~11,12~21,income1 22~31,32~41; income2 42~52,52~61

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

/Users/fxr/opt/anaconda3/lib/python3.9/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/Users/fxr/opt/anaconda3/lib/python3.9/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/Users/fxr/opt/anaconda3/lib/python3.9/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/Users/fxr/opt/anaconda3/lib/python3.9/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
